In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import seaborn as sns
import os
from pymc_marketing.mmm.transformers import geometric_adstock, logistic_saturation
from pymc_marketing.mmm.delayed_saturated_mmm import DelayedSaturatedMMM
from pymc_marketing.mmm.utils import generate_fourier_modes

az.style.use("arviz-darkgrid")
plt.rcParams["figure.figsize"] = [12, 7]
plt.rcParams["figure.dpi"] = 100

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

In [29]:
os.chdir('C:/Users/norri/Desktop/')
df = pd.read_csv('pymc_data.csv')
# seed: int = sum(map(ord, "mmm"))
# rng: np.random.Generator = np.random.default_rng(seed=seed)
#
# # date range
# min_date = pd.to_datetime("2018-04-01")
# max_date = pd.to_datetime("2021-09-01")
#
# df = pd.DataFrame(
#     data={"date_week": pd.date_range(start=min_date, end=max_date, freq="W-MON")}
# ).assign(
#     year=lambda x: x["date_week"].dt.year,
#     month=lambda x: x["date_week"].dt.month,
#     dayofyear=lambda x: x["date_week"].dt.dayofyear,
# )
#
n = df.shape[0]
# print(f"Number of observations: {n}")



Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\norri\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\norri\AppData\Local\Temp\ipykernel_2552\320828814.py", line 2, in <cell line: 2>
    df = pd.read_csv('pymc_data.csv')
  File "C:\Users\norri\.conda\envs\pymc_env\lib\site-packages\pandas\io\parsers\readers.py", line 912, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "C:\Users\norri\.conda\envs\pymc_env\lib\site-packages\pandas\io\parsers\readers.py", line 583, in _read
    return parser.read(nrows)
  File "C:\Users\norri\.conda\envs\pymc_env\lib\site-packages\pandas\io\parsers\readers.py", line 1721, in read
    df = DataFrame(col_dict, columns=columns, index=index)
  File "C:\Users\norri\.conda\envs\pymc_env\lib\site-packages\pandas\core\frame.py", line 708, in __init__
    mgr = dict_to_mgr(data, index, columns, dtype=dtype, copy=copy, t

In [30]:
# media data
# x1 = df.uniform(low=0.0, high=1.0, size=n)
# df["x1"] = np.where(x1 > 0.9, x1, x1 / 2)
#
# x2 = df.uniform(low=0.0, high=1.0, size=n)
# df["x2"] = np.where(x2 > 0.8, x2, 0)


# fig, ax = plt.subplots(
#     nrows=2, ncols=1, figsize=(10, 7), sharex=True, sharey=True, layout="constrained"
# )
# sns.lineplot(x="date_week", y="x1", data=df, color="C0", ax=ax[0])
# sns.lineplot(x="date_week", y="x2", data=df, color="C1", ax=ax[1])
# ax[1].set(xlabel="date")
# fig.suptitle("Media Costs Data", fontsize=16);

In [31]:
# apply geometric adstock transformation
alpha1: float = 0.4
alpha2: float = 0.2

df["x1_adstock"] = (
    geometric_adstock(
        x=df["x1"].to_numpy(), alpha=alpha1, l_max=8, normalize=True
    )
    .eval()
    .flatten()
)

df["x2_adstock"] = (
    geometric_adstock(
        x=df["x2"].to_numpy(), alpha=alpha2, l_max=8, normalize=True
    )
    .eval()
    .flatten()
)

KeyError: 'x1'

In [32]:
# apply saturation transformation
lam1: float = 4.0
lam2: float = 3.0

df["x1_adstock_saturated"] = logistic_saturation(
    x=df["x1_adstock"].to_numpy(), lam=lam1
).eval()

df["x2_adstock_saturated"] = logistic_saturation(
    x=df["x2_adstock"].to_numpy(), lam=lam2
).eval()

KeyError: 'x1_adstock'

In [33]:
fig, ax = plt.subplots(
    nrows=3, ncols=2, figsize=(16, 9), sharex=True, sharey=False, layout="constrained"
)
sns.lineplot(x="date_week", y="x1", data=df, color="C0", ax=ax[0, 0])
sns.lineplot(x="date_week", y="x2", data=df, color="C1", ax=ax[0, 1])
sns.lineplot(x="date_week", y="x1_adstock", data=df, color="C0", ax=ax[1, 0])
sns.lineplot(x="date_week", y="x2_adstock", data=df, color="C1", ax=ax[1, 1])
sns.lineplot(x="date_week", y="x1_adstock_saturated", data=df, color="C0", ax=ax[2, 0])
sns.lineplot(x="date_week", y="x2_adstock_saturated", data=df, color="C1", ax=ax[2, 1])
fig.suptitle("Media Costs Data - Transformed", fontsize=16);

TypeError: float() argument must be a string or a real number, not '_NoValueType'

<Figure size 1600x900 with 0 Axes>

In [34]:
df["trend"] = (np.linspace(start=0.0, stop=50, num=n) + 10) ** (1 / 4) - 1

df["cs"] = - np.sin(2 * 2 * np.pi * df["dayofyear"] / 365.5)
df["cc"] = np.cos(1 * 2 * np.pi * df["dayofyear"] / 365.5)
df["seasonality"] = 0.5 * (df["cs"] + df["cc"])

fig, ax = plt.subplots()
sns.lineplot(x="date_week", y="trend", color="C2", label="trend", data=df, ax=ax)
sns.lineplot(x="date_week", y="seasonality", color="C3", label="seasonality", data=df, ax=ax)
ax.legend(loc="upper left")
ax.set(title="Trend & Seasonality Components", xlabel="date", ylabel=None);

ValueError: Length of values (179) does not match length of index (200)

In [24]:
df["event_1"] = (df["date_week"] == "2019-05-13").astype(float)
df["event_2"] = (df["date_week"] == "2020-09-14").astype(float)

KeyError: 'date_week'

In [25]:
df["intercept"] = 2.0
df["epsilon"] = rng.normal(loc=0.0, scale=0.25, size=n)

amplitude = 1
beta_1 = 3.0
beta_2 = 2.0
betas = [beta_1, beta_2]


df["y"] = amplitude * (
        df["intercept"]
        + df["trend"]
        + df["seasonality"]
        + 1.5 * df["event_1"]
        + 2.5 * df["event_2"]
        + beta_1 * df["x1_adstock_saturated"]
        + beta_2 * df["x2_adstock_saturated"]
        + df["epsilon"]
)

fig, ax = plt.subplots()
sns.lineplot(x="date_week", y="y", color="black", data=df, ax=ax)
ax.set(title="Sales (Target Variable)", xlabel="date", ylabel="y (thousands)");

ValueError: Length of values (179) does not match length of index (200)

In [ ]:
contribution_share_x1: float = (
        (beta_1 * df["x1_adstock_saturated"]).sum()
        / (beta_1 * df["x1_adstock_saturated"] + beta_2 * df["x2_adstock_saturated"]).sum()
)

contribution_share_x2: float = (
        (beta_2 * df["x2_adstock_saturated"]).sum()
        / (beta_1 * df["x1_adstock_saturated"] + beta_2 * df["x2_adstock_saturated"]).sum()
)

print(f"Contribution Share of x1: {contribution_share_x1:.2f}")
print(f"Contribution Share of x2: {contribution_share_x2:.2f}")

In [ ]:
fig, ax = plt.subplots(
    nrows=2, ncols=1, figsize=(12, 8), sharex=True, sharey=False, layout="constrained"
)

for i, x in enumerate(["x1", "x2"]):
    sns.scatterplot(
        x=df[x],
        y=amplitude * betas[i] * df[f"{x}_adstock_saturated"],
        color=f"C{i}",
        ax=ax[i],
    )
    ax[i].set(
        title=f"$x_{i + 1}$ contribution",
        ylabel=f"$\\beta_{i + 1} \cdot x_{i + 1}$ adstocked & saturated",
        xlabel="x",
    )

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
df[["x1", "x2"]].sum().plot(kind="bar", color=["C0", "C1"], ax=ax)
ax.set(title="Total Media Spend", xlabel="Media Channel", ylabel="Costs (thousands)");

In [ ]:
roas_1 = (amplitude * beta_1 * df["x1_adstock_saturated"]).sum() / df["x1"].sum()
roas_2 = (amplitude * beta_2 * df["x2_adstock_saturated"]).sum() / df["x2"].sum()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
(
    pd.Series(data=[roas_1, roas_2], index=["x1", "x2"])
    .plot(kind="bar", color=["C0", "C1"])
)

ax.set(title="ROAS (Approximation)", xlabel="Media Channel", ylabel="ROAS");